In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import chess
import nbimporter
import sys
from typing import Any, Callable
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI

# Aufgabe 06: Minimax mit Alpha-Beta-Pruning und Memoisierung

Dieses Notebook implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung.

In [ ]:
class Exercise06AI(Exercise05AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning and memoization."""
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, tuple[str, int, chess.Move]] = {}

Wie auch schon in der Elternklasse `Exercise05AI` existiert eine Funktion `reset`, die dazu dient, die Instanz nach einem beendeten Spiel in ihren Ursprungszustand zurückzusetzen. Diese hat dieselbe Funktion wie die der Elternklasse, leert zusätzlich aber noch den implementierten Cache.

In [ ]:
class Exercise06AI(Exercise06AI):  # type: ignore
    def reset(self) -> None:
        """Resets all internal variables."""
        super().reset()
        self.cache.clear()

## Memoisierung

Wie auch in anderen Spielen kann im Schach eine Position mehrfach auftreten. Die Sequenz von Zügen welche zu einer solchen Position führt, wird Transposition genannt [3]. In diesem Fall gibt es eine andere Sequenz welche zu derselben Position führt. Hieraus folgt, dass es weniger eindeutige Positionen als verschiedene Spielsequenzen gibt.  Dieser Umstand kann nun genutzt werden, um die Anzahl der zu untersuchenden Knoten im Suchbaum durch die sog. Memoisierung zu verringern [2]. Hierbei wird das Ergebnis von jedem berechneten Knoten mit der ausgehenden Position und der Suchtiefe als Identifikator in einem Cache gespeichert. Wenn bei der Berechnung nun dieselbe Position erneut auftritt und die Suchtiefe ebenfalls übereinstimmt, kann das Ergebnis aus dem Cache verwendet werden und wird nicht neu berechnet. Dieses Vorgehen beschleunigt die Suche insgesamt, da nun weniger Knoten untersucht werden müssen, führt aber auch zu einem höheren Speicherverbrauch.

### Memoisierung bei Alpha-Beta-Pruning

Prinzipiell wäre es möglich, die Memoisierung auch mit vorhandenem Alpha-Beta-Pruning genau so durchzuführen wie oben beschrieben. In diesem Fall müssten die Werte für Alpha und Beta als Identifikator mit aufgenommen werden. Da sich diese Werte aber sehr oft ändern, gäbe es nur sehr selten Übereinstimmungen innerhalb der Suche und der Geschwindigkeitsvorteil wäre nur sehr klein [2]. Stattdessen wird ein anderer Ansatz gewählt [2]:

Anstatt nur das Ergebnis einer Berechnung zu speichern wird nun zusätzlich ein Erkennungszeichen $\textrm{flag} \in \{ `\textrm{≤}`, `\textrm{=}`, `\textrm{≥}` \} $ hinzugefügt. Für ein berechnetes Ergebnis $v, m = \textrm{minimax}(s, d, \alpha, \beta)$ wird also das Tripel $(\textrm{flag}, v, m)$ im Cache abgelegt. Hierbei ist $v \in \mathbb{Z}$ die ermittelte Bewertung der Suche, $m$ der berechnete beste Zug, $\textrm{minimax}$ die nicht-memoisierte Suchfunktion mit Alpha-Beta-Pruning, $s$ der aktuelle Spielzustand (Board), $d \in \mathbb{N}$ die gewählte Suchtiefe und $\alpha, \beta \in \mathbb{Z}$ sind die Grenzen des Alpha-Beta-Prunings. Der Identifikator für den Cache bleibt weiterhin wie oben beschrieben, also ohne die Werte Alpha und Beta.

Insgesamt erfüllen die Werte aus dem Cache für einen gegebenen Identifikator $(s, d)$ nun die folgende Spezifikation:
$$
\begin{align*}
&\textrm{Cache}[s,d] = (`\textrm{=}`, v, m) &&\implies \textrm{minimax}(s, d, \alpha, \beta) = v, m \\
&\textrm{Cache}[s,d] = (`\textrm{≤}`, v_1, m) &&\implies \textrm{minimax}(s, d, \alpha, \beta) = v_2, m \; \land \; v_2 \le v_1 \\
&\textrm{Cache}[s,d] = (`\textrm{≥}`, v_1, m) &&\implies \textrm{minimax}(s, d, \alpha, \beta) = v_2, m \; \land \; v_2 \ge v_1
\end{align*}
$$

Die konkrete Berechnung der Zustände erfolgt nun mit der Funktion $\textrm{memoized_minimax}$, welche dieselbe Signatur wie die Funktion $\textrm{minimax}$ besitzt. Wenn für einen gegebenen Identifikator $(s, d)$ ein Eintrag im Cache vorhanden ist, erfolgt die Berechnung $\textrm{memoized_minimax}(s, d, \alpha, \beta)$ nach der folgenden Fallunterscheidung:
$$
\begin{align*}
&\textrm{Cache}[s,d] = (`\textrm{=}`, v, m)                          &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = v, m \\
&\textrm{Cache}[s,d] = (`\textrm{≤}`, v, m) \land v \le \alpha       &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = v, m \\
&\textrm{Cache}[s,d] = (`\textrm{≤}`, v, m) \land \alpha < v < \beta &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = \textrm{minimax}(s, d, \alpha, v) \\
&\textrm{Cache}[s,d] = (`\textrm{≤}`, v, m) \land \beta \le v        &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = \textrm{minimax}(s, d, \alpha, \beta) \\
&\textrm{Cache}[s,d] = (`\textrm{≥}`, v, m) \land v \ge \beta        &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = v, m \\
&\textrm{Cache}[s,d] = (`\textrm{≥}`, v, m) \land \alpha < v < \beta &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = \textrm{minimax}(s, d, v, \beta) \\
&\textrm{Cache}[s,d] = (`\textrm{≥}`, v, m) \land v \le \alpha       &&\implies \textrm{memoized_minimax}(s, d, \alpha, \beta) = \textrm{minimax}(s, d, \alpha, \beta) \\
\end{align*}
$$

## Memoisierungsfunktion

Die Klasse wird um eine Variable `cache` erweitert. Initial ist `cache` ein leeres Dictionary, welches als Schlüssel eine Repräsentation eines Boards als Tupel annimmt und als Wert ein Tripel bestehend aus einem einzelnen Erkennungszeichen, einer Board-Beurteilung und einem Zug zurückgibt.

Die Memoisierungsfunktion `memoize_minimax` nimmt die Minimax-Funktion `minimax` als Argument und gibt eine memoisierte Version `minimax_memoized` dieser Funktion zurück.

Die memoisierte Funktion `minimax_memoized` versucht zunächst, den Rückgabewert der Ursprungsfunktion aus dem `cache` auszulesen. Wenn der `cache` ein Ergebnis für den berechneten Identifikator enthält, wird die Hilfsfunktion `get_from_cache` aufgerufen und das Ergebnis zurückgegeben. Andernfalls wird die Ursprungsfunktion `minimax` aufgerufen, der Rückgabewert wird für die verwendeten Argumente im `cache` hinterlegt und schlussendlich wird der berechnete Wert zurückgegeben. Zum Speichern des neuen Rückgabewertes wird die Hilfsfunktion `store_in_cache` verwendet.  
Der Cache wird bei jedem Bauernzug, Schlagen einer Figur und veränderten Rochade- oder En-Passant-Rechten zurückgesetzt, da in diesem Fall die meisten der zuvor berechneten Stellungen nicht wieder auftreten können und dies den Speicherbedarf des Programms verringert.

Da die Memoisierungsfunktion in den weiteren ChessAI-Versionen ebenfalls verwendet werden soll, wird diese als Dekorator-Funktion implementiert.

In [ ]:
class Exercise06AI(Exercise06AI):  # type: ignore
    @staticmethod
    def memoize_minimax(minimax: Callable):
        def minimax_memoized(
            self,
            board: chess.Board,
            depth: int,
            current_evaluation: int,
            alpha: int = -Exercise06AI.LIMIT,
            beta: int = Exercise06AI.LIMIT,
        ):
            key = Exercise04AI.get_key(board, depth)
            self.stats[-1]["cache_tries"] += 1
            if key in self.cache:
                self.stats[-1]["cache_hits"] += 1
                return self.get_from_cache(minimax, key, board, depth, current_evaluation, alpha, beta)
            result = minimax(self, board, depth, current_evaluation, alpha, beta)
            self.store_in_cache(key, result, alpha, beta)
            return result

        return minimax_memoized

Die Funktion `minimax` ruft die Minimax Funktion der Elternklasse `Exercise05AI` auf, wendet allerdings die implementierte Dekoratorfunktion darauf an, wodurch die Minimax Funktion um die Memoisierung erweitert wird.

In [ ]:
class Exercise06AI(Exercise06AI):  # type: ignore
    @Exercise06AI.memoize_minimax
    def minimax(self, *args) -> int:
        """Memoized version of the Exercise05AI minimax implementation."""
        return super().minimax(*args)

### Speicherung im Cache

Das Speichern der Ergebnisse wird von der Funktion `store_in_cache` übernommen. Diese bekommt als Argumente den Cache-Identifikator des aktuellen Boards (`key`), das berechnete Minimax-Ergebnis als Tupel (`result`), die Grenzwerte $\textrm{alpha}$ und $\textrm{beta}$ und speichert das Ergebnis im Cache ab. Hierbei werden die oben vorgestellten Invarianten beachtet.

In [ ]:
class Exercise06AI(Exercise06AI):  # type: ignore
    def store_in_cache(self, key: tuple, result: tuple, alpha: int, beta: int) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation <= alpha:
            self.cache[key] = ("≤", evaluation, move)
        elif evaluation < beta:
            self.cache[key] = ("=", evaluation, move)
        else:
            self.cache[key] = ("≥", evaluation, move)

### Abrufen aus dem Cache

Die Funktion `get_from_cache` nimmt als Parameter die Funktion `minimax`, den Cache-Identifikator für das aktuelle Board (`key`), sowie alle Parameter der Funktion `minimax` entgegen und gibt ein Tupel der Form $(\textrm{evaluation}, \textrm{move})$ zurück. Zunächst wird das Tripel $ (\textrm{flag}, \textrm{evaluation}, \textrm{move}) $ mit dem gegebenen `key` aus dem Cache extrahiert. Anschließend erfolgt die oben vorgestellte Fallunterscheidung.

In [ ]:
class Exercise06AI(Exercise06AI):  # type: ignore
    def get_from_cache(
        self,
        minimax: Callable,
        key: tuple,
        board: chess.Board,
        depth: int,
        current_eval: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Gets a result from the cache if possible."""
        flag, evaluation, move = self.cache[key]
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(self, board, depth, current_eval, alpha, evaluation)
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(self, board, depth, current_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(self, board, depth, current_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(self, board, depth, current_eval, evaluation, beta)
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move

Die Funktion `get_next_middle_game_move` ruft die gleichnamige Funktion der Elternklasse auf, speichert aber zusätzliche, auf den Cache bezogene, Metriken in dem `stats` Dictionary, welche zur Beurteilung der Memoisierung verwendet werden.

In [ ]:
class Exercise06AI(Exercise06AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move."""
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["cache_hits"] = 0
        next_move = super().get_next_middle_game_move(board)
        if board.is_irreversible(next_move):
            self.cache.clear()
            self.stats[-1]["cache_cleared"] = True
        else:
            self.stats[-1]["cache_cleared"] = False
        self.stats[-1]["cache_size_mb"] = round(sys.getsizeof(self.cache) / (1024 * 1024), 2)
        return next_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI
import Exercise02AI as Exercise02AI_
import Exercise04AI as Exercise04AI_

In [ ]:
# Create player and board
unit_test_player = Exercise06AI(player_name="Ex06AI", search_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

Um die Funktionsweise der Methode `store_in_cache` zu validieren, wird ein statisches Ergebnis mit dem Evaluierungswert $10$ im Cache abgespeichert. Die `store_in_cache`-Funktion wird hierbei dreimal mit unterschiedlichen $alpha$ und $beta$ Werten aufgerufen. Getestet wird hierbei, ob für jeden Aufruf die richtige $	extrm{flag}$ gespeichert wird. Es gilt:

$$
\textrm{flag} = \begin{cases}
`\textrm{≤}` \texttt{ g.d.w. } \textrm{evaluation} <= \textrm{alpha} \\
`\textrm{=}` \texttt{ g.d.w. } \textrm{alpha} < \textrm{evaluation} < \textrm{beta} \\
`\textrm{≥}` \texttt{ sonst.}
\end{cases}
$$

In [ ]:
# Test cache store function
def test_store_in_cache(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache = {}  # Clear cache
    test_result = (10, chess.Move.from_uci("d4c3"))
    test_key_1 = ("test", 1)
    unit_test_player.store_in_cache(test_key_1, test_result, alpha=5, beta=15)
    test_key_2 = ("test", 2)
    unit_test_player.store_in_cache(test_key_2, test_result, alpha=12, beta=15)
    test_key_3 = ("test", 3)
    unit_test_player.store_in_cache(test_key_3, test_result, alpha=5, beta=8)
    print(f"Cache: {unit_test_player.cache}")
    assert (
        unit_test_player.cache[test_key_1] == ("=", *test_result)
    ), "Cache result for '=' does not match expected value!"
    assert (
        unit_test_player.cache[test_key_2] == ("≤", *test_result)
    ),"Cache result for '≤' does not match expected value!"
    assert (
        unit_test_player.cache[test_key_3] == ("≥", *test_result)
    ), "Cache result for '≥' does not match expected value!"

In [ ]:
test_store_in_cache(unit_test_player, board)

Um den Minimax mit Alpha-Beta-Pruning und Memoisierung zu testen, wird die Testmethode der `Exercise04AI` erneut verwendet, da diese den Cache berücksichtigt. Der Funktionsaufruf erfolgt mit denselben Evaluierungswerten, allerdings mit reduzierten Metriken, da die Memoisierung die Anzahl der berechneten Knoten vermindert.

In [ ]:
# Test minimax
Exercise04AI_.test_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=2336,
    expected_cache_tries=2788,
    expected_cache_hits=557,
    expected_cache_elements=2231,
)

Um zu validieren, dass die Memoisierung das gewünschte Ergebnis zurückliefert, wird die memoisierte Minimax-Testfunktion der `Exercise04AI` mit denselben Evaluierungsparametern wie zuvor aufgerufen. Das Ergebnis der Funktion soll dasselbe sein, es soll jedoch nur einmal auf den Cache zugegriffen werden, wodurch direkt das richtige Ergebnis zurückgeliefert werden soll.

In [ ]:
# Test minimax again (now memoized)
Exercise04AI_.test_memoized_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=0,
    expected_cache_tries=1,
    expected_cache_hits=1,
    expected_cache_elements=2231,
)

Um die Funktion `get_next_middle_game_move` zu testen, wird überprüft, ob diese korrekterweise den Zug zurückgibt, in welchem die Dame durch den schwarzen Bauern geschlagen wird. Durch die implementierte Memoisierung sollen hierbei keine neuen Knoten untersucht werden.

In [ ]:
# Test next move function (with memoized minimax result)
Exercise02AI_.test_next_move(
    unit_test_player,
    board,
    expected_move="d4c3",
    expected_nodes=0,
)

Abschließend wird die `reset`-Funktion analog zur `Exercise04AI` getestet.

In [ ]:
# Test reset function
Exercise04AI_.test_reset(unit_test_player, board)